# Setup

I'm using a separate version of python (maybe I shouldn't) and the only way I got this to work was to cd into the source directory.

In [2]:
cd /home/amemory/opt/Trio-1.1.3/trio/

/home/amemory/opt/Trio-1.1.3/trio


In [3]:
pwd

u'/home/amemory/opt/Trio-1.1.3/trio'

In [4]:
!ls

common.py  __init__.py	parser	sql	  triodb.py   visitor
executor   node.py	spi	trio.cfg  triodb.pyc  xtuple.py


In [5]:
import triodb

In [6]:
conn = triodb.connect(database='trio1', 
                      user='amemory', 
                      password='password')

In [7]:
#conn.close()

It seems that the DB always starts empty.

In [8]:
cur = conn.cursor()

# Single tuple

Create a single database, insert a single tuple, then query that tuple and see its lineage.

In [23]:
cur.execute("create table CRIME(type varchar(32), sector int)")

In [24]:
cur.execute("""insert into CRIME values ('Kidnapping', 1)""")

In [25]:
cur.execute("select * from crime")
tlst = cur.xfetchall()

In [27]:
tup = tlst[0]
tup

<trio.xtuple.XTuple instance at 0x7fafeac82830>

In [28]:
alt = tup.getAlternative(0)
alt

<trio.xtuple.Alternative instance at 0x7fafeac82bd8>

In [29]:
alt.getConfidence()

Not sure why there is no confidence and the computeConfidence call experiences an exception.

In [30]:
alt.computeConfidence(conn)

'error'

In [31]:
lin = alt.getLineage()
lin

<trio.xtuple.Lineage instance at 0x7fafeac82998>

In [32]:
lin.items()

[('crime', 1, 0)]

It seems that this lineage refers to id 1 of the crime relation.

In [33]:
lin.aids()

[1]

In [34]:
lin.aidstrs()

['1']

# Crime

In [50]:
conn.close()

In [51]:
conn = triodb.connect(database='trio1', 
                      user='amemory', 
                      password='password')
cur = conn.cursor()

In [40]:
# Causes an error -- the DB is empty again after closing and reopening
# cur.execute("select * from crime")
# tlst = cur.xfetchall()

## Load crime

In [52]:
sql = """
create table CRIME(type varchar(32), sector int);
insert into CRIME values ('Kidnapping', 1); 
insert into CRIME values ('Car theft', 2); 
insert into CRIME values ('Robbery', 2); 
create table WITNESS(name varchar(32), sector int);
insert into WITNESS values ('Mary', 2);
insert into WITNESS values ('Susan', 1);
insert into WITNESS values ('Jane', 1);
create table PERSON(name varchar(32), hair varchar(32), clothes varchar(32));
insert into PERSON values ('Jim', 'black', 'red');
insert into PERSON values ('Hank', 'brown', 'green');
insert into PERSON values ('Bob', 'gray', 'blue');
create trio table SAWPERSON(witness varchar(32), hair varchar(32), clothes varchar(32), uncertain(hair, clothes)); 
insert into SAWPERSON values ('Mary', 'black', ['red' | 'green']);
insert into SAWPERSON values ('Jane', 'gray', 'blue');
insert into SAWPERSON values ('Jane', 'brown', 'green') ?;
insert into SAWPERSON values [('Susan', 'gray', 'red') |('Susan', 'brown', 'red') |('Susan', 'brown', 'green')];
create trio table DRIVES(person varchar(32), car varchar(32), uncertain(person, car)) with confidences;
insert into DRIVES values [('Jim', 'Honda'):1];
insert into DRIVES values (['Hank':0.6 | 'Jim':0.3], 'Toyota':1);
insert into DRIVES values ('Bob':1, ['Toyota':0.5 | 'Mazda':0.3]);
create trio table SAWCAR(witness varchar(32), car varchar(32), uncertain(car)) with confidences;
insert into SAWCAR values [('Susan', 'Honda') :1];
insert into SAWCAR values ('Mary':1, 'Toyota':0.7);
insert into SAWCAR values [ ('Jane', 'Toyota' ) :0.5 | ('Jane', 'Honda') :0.4];
"""

In [53]:
for line in sql.split(";"):
    line = line.strip()
    line = line.replace("\n"," ")
    print line
    if line == "":
        continue
    cur.execute(line)

create table CRIME(type varchar(32), sector int)
insert into CRIME values ('Kidnapping', 1)
insert into CRIME values ('Car theft', 2)
insert into CRIME values ('Robbery', 2)
create table WITNESS(name varchar(32), sector int)
insert into WITNESS values ('Mary', 2)
insert into WITNESS values ('Susan', 1)
insert into WITNESS values ('Jane', 1)
create table PERSON(name varchar(32), hair varchar(32), clothes varchar(32))
insert into PERSON values ('Jim', 'black', 'red')
insert into PERSON values ('Hank', 'brown', 'green')
insert into PERSON values ('Bob', 'gray', 'blue')
create trio table SAWPERSON(witness varchar(32), hair varchar(32), clothes varchar(32), uncertain(hair, clothes))
insert into SAWPERSON values ('Mary', 'black', ['red' | 'green'])
insert into SAWPERSON values ('Jane', 'gray', 'blue')
insert into SAWPERSON values ('Jane', 'brown', 'green') ?
insert into SAWPERSON values [('Susan', 'gray', 'red') |('Susan', 'brown', 'red') |('Susan', 'brown', 'green')]
create trio table DRIVE

## Crime views

In [56]:
sql = """
create table SUSPECT_S as
select PERSON.name as suspect, CRIME.type as crime
from  SAWPERSON, WITNESS, CRIME, PERSON
where SAWPERSON.witness = WITNESS.name
  and CRIME.sector = WITNESS.sector 
  and PERSON.hair = SAWPERSON.hair
  and PERSON.clothes = SAWPERSON.clothes;

create table SUSPECT_D as 
select DRIVES.person as suspect, CRIME.type as crime
from  SAWCAR, WITNESS, CRIME, DRIVES
where SAWCAR.witness = WITNESS.name
  and WITNESS.sector = CRIME.sector
  and DRIVES.car = SAWCAR.car; 

create table HIGH_SUSPECT as
select SUSPECT_D.suspect, SUSPECT_D.crime
from SUSPECT_D, SUSPECT_S
where SUSPECT_D.suspect = SUSPECT_S.suspect 
  and SUSPECT_D.crime = SUSPECT_S.crime
  and conf(SUSPECT_D) >= 0.25;

create table ACCOMPLICES as 
select S1.suspect as name1, S2.suspect as name2 
from SUSPECT_D S1, SUSPECT_D S2
where S1.crime = S2.crime
  and S1.suspect < S2.suspect;
"""

In [57]:
for line in sql.split(";"):
    line = line.strip()
    line = line.replace("\n"," ")
    print line
    if line == "":
        continue
    cur.execute(line)

create table SUSPECT_S as select PERSON.name as suspect, CRIME.type as crime from  SAWPERSON, WITNESS, CRIME, PERSON where SAWPERSON.witness = WITNESS.name   and CRIME.sector = WITNESS.sector    and PERSON.hair = SAWPERSON.hair   and PERSON.clothes = SAWPERSON.clothes
SELECT: 0.00502395629883
INDEX:  0
create table SUSPECT_D as  select DRIVES.person as suspect, CRIME.type as crime from  SAWCAR, WITNESS, CRIME, DRIVES where SAWCAR.witness = WITNESS.name   and WITNESS.sector = CRIME.sector   and DRIVES.car = SAWCAR.car
SELECT: 0.00916695594788
INDEX:  0
create table HIGH_SUSPECT as select SUSPECT_D.suspect, SUSPECT_D.crime from SUSPECT_D, SUSPECT_S where SUSPECT_D.suspect = SUSPECT_S.suspect    and SUSPECT_D.crime = SUSPECT_S.crime   and conf(SUSPECT_D) >= 0.25
SELECT: 0.00465202331543
INDEX:  0
select suspect, crime from HIGH_SUSPECT H, WITNESS W where H ==> W and W.name = 'Susan'
create table ACCOMPLICES as  select S1.suspect as name1, S2.suspect as name2  from SUSPECT_D S1, SUSPECT_D 

## Query

In [58]:
sql = """
select suspect, crime
from HIGH_SUSPECT H, WITNESS W
where H ==> W
and W.name = 'Susan';
"""

In [65]:
def q(sql):
    return sql.replace("\n"," ").replace(";","")

In [67]:
cur.execute("select suspect, crime from HIGH_SUSPECT H, WITNESS W where H ==> W and W.name = 'Susan'")
tlst = cur.xfetchall()

In [72]:
tup = tlst[0]
tup

<trio.xtuple.XTuple instance at 0x7fafeac6be18>

In [75]:
alt = tup.alternatives[0]
alt

<trio.xtuple.Alternative instance at 0x7fafeac70320>

In [76]:
lin = alt.getLineage()
lin

<trio.xtuple.Lineage instance at 0x7fafeac6b950>

In [78]:
lin.items()

[('high_suspect', 2, 0), ('witness', 2, 0)]

Not sure this lineage makes sense yet, but it seems to be a conjunction of the 2nd tuple of high_suspect and the 2nd tuple of witness.